In [ ]:
import torch 
import torch.nn as nn
import numpy as np
from torchvision import transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import torch.optim as optim

from torch.utils.tensorboard import SummaryWriter

In [ ]:
from typing import Any
import numpy 
class ItemLoader():
    def __init__(self) -> None:
        pass
    def __call__(self, file_path) -> Any:
        return np.loadtxt(fname=file_path,dtype=numpy.float32,delimiter=",")
tran = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize(size=(128,128)),
    transforms.Normalize(mean=(0.5),std=(0.5))
    ])
batch_size = 64
train_dataset = datasets.DatasetFolder(root = "./dataset/", transform=tran,loader=ItemLoader(),extensions=[".csv"])
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)


In [ ]:
from models import NoiseNet
model = NoiseNet()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.1)


In [ ]:
def train(epoch):
    running_loss = 0.0
    for batch_idx, data in enumerate(train_loader, 0):
        # 获得一个批次的数据和标签
        inputs, target = data
        optimizer.zero_grad()
        # 获得模型预测结果
        outputs = model(inputs)
        # 交叉熵代价函数outputs(64,10),target（64）
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()
 
        running_loss += loss.item()
    return running_loss
 

In [ ]:
if __name__ == '__main__':
    min_loss = 2.0
    loss_list = []
    
    writer = SummaryWriter()
    for epoch in range(1000):
        loss = train(epoch)
        writer.add_scalar("epoch:loss",loss,epoch)
        loss_list.append(loss_list)
        if(min_loss > loss):
            min_loss = loss
            torch.save({
                "epoch":epoch,
                "loss":loss,
                "optimizer":optimizer.state_dict(),
                "module":model.state_dict()
            },"./noise.plt")
        print("[%d]LOSS %.3f"%(epoch,loss))
    writer.close()
    plt.plot(loss_list)